In [26]:
# -*- coding: utf-8 -*-
"""Untitled10.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1nUBL6MYUbP-CAumzQhbamevEKKNGVBU8
"""

import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model


def _conv_block(inp, convs, skip=True):
	x = inp
	c = 0
	for conv in convs:
		if c == (len(convs) - 2) and skip:
			skip_connection = x
		c += 1
		if conv['s'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
		x = Conv2D(conv['f'],
				   conv['k'],
				   strides=conv['s'],
				   padding='valid' if conv['s'] > 1 else 'same', # peculiar padding as darknet prefer left and top
				   name='conv_' + str(conv['layers']),
				   use_bias=False if conv['batch'] else True)(x)
		if conv['batch']: x = BatchNormalization(epsilon=0.001, name='batch_' + str(conv['layers']))(x)
		if conv['relu']: x = LeakyReLU(alpha=0.1, name='relu_' + str(conv['layers']))(x)
	return add([skip_connection, x]) if skip else x

def make_model():
	img_inp = Input(shape=(None, None, 3))
	
	x = _conv_block(img_inp, [{'f': 32, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 0},
								  {'f': 64, 'k': 3, 's': 2, 'batch': True, 'relu': True, 'layers': 1},
								  {'f': 32, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 2},
								  {'f': 64, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 3},
									{'f': 32, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 4}])
	
	x = _conv_block(x, [{'f': 128, 'k': 3, 's': 2, 'batch': True, 'relu': True, 'layers': 5},
						{'f':  64, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 6},
						{'f': 128, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 7}])
	
	x = _conv_block(x, [{'f':  64, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 9},
						{'f': 128, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 10}])
	
	x = _conv_block(x, [{'f': 256, 'k': 3, 's': 2, 'batch': True, 'relu': True, 'layers': 12},
						{'f': 128, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 13},
						{'f': 256, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 14}])
	
	for i in range(7):
		x = _conv_block(x, [{'f': 128, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 16+i*3},
							{'f': 256, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 17+i*3}])
	skip_36 = x
	
	x = _conv_block(x, [{'f': 512, 'k': 3, 's': 2, 'batch': True, 'relu': True, 'layers': 37},
						{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 38},
						{'f': 512, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 39}])
	
	for i in range(7):
		x = _conv_block(x, [{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 41+i*3},
							{'f': 512, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 42+i*3}])
	skip_61 = x
	
	x = _conv_block(x, [{'f': 1024, 'k': 3, 's': 2, 'batch': True, 'relu': True, 'layers': 62},
						{'f':  512, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 63},
						{'f': 1024, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 64}])
	
	for i in range(3):
		x = _conv_block(x, [{'f':  512, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 66+i*3},
							{'f': 1024, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 67+i*3}])
	
	x = _conv_block(x, [{'f':  512, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 75},
						{'f': 1024, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 76},
						{'f':  512, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 77},
						{'f': 1024, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 78},
						{'f':  512, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 79}], skip=False)
	
	yolo_82 = _conv_block(x, [{'f': 1024, 'k': 3, 's': 1, 'batch': True,  'relu': True,  'layers': 80},
							  {'f':  255, 'k': 1, 's': 1, 'batch': False, 'relu': False, 'layers': 81}], skip=False)
	
	x = _conv_block(x, [{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 84}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_61])
	
	x = _conv_block(x, [{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 87},
						{'f': 512, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 88},
						{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 89},
						{'f': 512, 'k': 3, 's': 1, 'batch': True, 'relu': True, 'layers': 90},
						{'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True, 'layers': 91}], skip=False)
	
	yolo_94 = _conv_block(x, [{'f': 512, 'k': 3, 's': 1, 'batch': True,  'relu': True,  'layers': 92},
							  {'f': 255, 'k': 1, 's': 1, 'batch': False, 'relu': False, 'layers': 93}], skip=False)
	
	x = _conv_block(x, [{'f': 128, 'k': 1, 's': 1, 'batch': True, 'relu': True,   'layers': 96},
	                    {'f': 256, 'k': 1, 's': 1, 'batch': True, 'relu': True,   'layers': 97}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_36])
	# Layer 99 => 106
	yolo_106 = _conv_block(x, [{'f': 128, 'k': 1, 's': 1, 'batch': True,  'relu': True,  'layers': 99},
							   {'f': 256, 'k': 3, 's': 1, 'batch': True,  'relu': True,  'layers': 100},
							   {'f': 128, 'k': 1, 's': 1, 'batch': True,  'relu': True,  'layers': 101},
							   {'f': 256, 'k': 3, 's': 1, 'batch': True,  'relu': True,  'layers': 102},
							   {'f': 128, 'k': 1, 's': 1, 'batch': True,  'relu': True,  'layers': 103},
							   {'f': 256, 'k': 3, 's': 1, 'batch': True,  'relu': True,  'layers': 104},
							   {'f': 255, 'k': 1, 's': 1, 'batch': False, 'relu': False, 'layers': 105}], skip=False)
	animal_model = Model(img_inp, [yolo_82, yolo_94, yolo_106])
	return animal_model

class Read_Weights:
	def __init__(self, weights):
		with open(weights, 'rb') as w:
			m,	= struct.unpack('i', w.read(4))
			n,	= struct.unpack('i', w.read(4))
			r, = struct.unpack('i', w.read(4))
			if (m*10 + n) >= 2 and m < 1000 and n < 1000:
				w.read(8)
			else:
				w.read(4)
			t = (m > 1000) or (n > 1000)
			b = w.read()
		self.off = 0
		self.all_weights = np.frombuffer(b, dtype='float32')
 
	def read_bytes(self, size):
		self.off = self.off + size
		return self.all_weights[self.off-size:self.off]
 
	def load_weights(self, model):
		for i in range(106):
			try:
				conv_layer = model.get_layer('conv_' + str(i))
				print("Weight Convo Number " + str(i))
				if i not in [81, 93, 105]:
					norm_layer = model.get_layer('batch_' + str(i))
					size = np.prod(norm_layer.get_weights()[0].shape)
					beta  = self.read_bytes(size) # bias
					gamma = self.read_bytes(size) # scale
					mean  = self.read_bytes(size) # mean
					var   = self.read_bytes(size) # variance
					weights = norm_layer.set_weights([gamma, beta, mean, var])
				if len(conv_layer.get_weights()) > 1:
					bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
					k = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					k = k.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					k = k.transpose([2,3,1,0])
					conv_layer.set_weights([k, bias])
				else:
					k = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					k = k.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					k = k.transpose([2,3,1,0])
					conv_layer.set_weights([k])
			except ValueError:
				print("No Convo Number " + str(i))
 
	def reset(self):
		self.off = 0

# define the model
animal_model = make_model()
# load the model weights
weights = Read_Weights('/content/drive/MyDrive/yolov3.weights')
# set the model weights into the model
weights.load_weights(animal_model)
# save the model to file
animal_model.save('animal.h5')

Weight Convo Number 0
Weight Convo Number 1
Weight Convo Number 2
Weight Convo Number 3
Weight Convo Number 4
Weight Convo Number 5
Weight Convo Number 6
Weight Convo Number 7
No Convo Number 8
Weight Convo Number 9
Weight Convo Number 10
No Convo Number 11
Weight Convo Number 12
Weight Convo Number 13
Weight Convo Number 14
No Convo Number 15
Weight Convo Number 16
Weight Convo Number 17
No Convo Number 18
Weight Convo Number 19
Weight Convo Number 20
No Convo Number 21
Weight Convo Number 22
Weight Convo Number 23
No Convo Number 24
Weight Convo Number 25
Weight Convo Number 26
No Convo Number 27
Weight Convo Number 28
Weight Convo Number 29
No Convo Number 30
Weight Convo Number 31
Weight Convo Number 32
No Convo Number 33
Weight Convo Number 34
Weight Convo Number 35
No Convo Number 36
Weight Convo Number 37
Weight Convo Number 38
Weight Convo Number 39
No Convo Number 40
Weight Convo Number 41
Weight Convo Number 42
No Convo Number 43
Weight Convo Number 44
Weight Convo Number 45
